In [25]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

import os
from pathlib import Path

folder = "colab/pytorch"
project_dir = "RF"

base_path = Path("/content/gdrive/My Drive/")
project_path = base_path / folder / project_dir
os.chdir(project_path)
for x in list(project_path.glob("*")):
    if x.is_dir():
        dir_name = str(x.relative_to(project_path))
        os.rename(dir_name, dir_name.split(" ", 1)[0])
print(f"현재 디렉토리 위치: {os.getcwd()}")
all_data = pd.read_csv('/content/gdrive/My Drive/colab/pytorch/RF/full_data.csv')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

sns.set(style="darkgrid")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
현재 디렉토리 위치: /content/gdrive/My Drive/colab/pytorch/RF


In [38]:
all_data

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
4976,Male,41.0,0,0,No,Private,Rural,70.15,29.8,formerly smoked,0
4977,Male,40.0,0,0,Yes,Private,Urban,191.15,31.1,smokes,0
4978,Female,45.0,1,0,Yes,Govt_job,Rural,95.02,31.8,smokes,0
4979,Male,40.0,0,0,Yes,Private,Rural,83.94,30.0,smokes,0


In [52]:
def preprocess(all_data):
  all_data = normalize(all_data)
  target = 'stroke'
  feature = ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
           'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
           'smoking_status']

  X = all_data.loc[:,all_data.columns!=target]
  y = all_data.loc[:,all_data.columns==target]

  X = pd.get_dummies(X[feature])

  oversample = SMOTE()
  X, y = oversample.fit_resample(X, y)
  X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size=0.20, shuffle=True)

  return X_train, X_test, Y_train, Y_test

In [53]:
def normalize(df):
    norm_feature = ['age', 'avg_glucose_level', 'bmi']
    result = df.copy()
    for feature_name in norm_feature:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [54]:
def rf(X_train, Y_train, n_estimators=250):
 BSPredictRF = RandomForestClassifier(n_estimators=n_estimators, n_jobs=-1)
 BSPredictRF.fit(X_train, Y_train)
 return BSPredictRF

In [55]:
X_train, X_test, Y_train, Y_test = preprocess(all_data)

In [59]:
BSPredictRF = rf(X_train,Y_train.values.ravel(), n_estimators=500)

In [60]:
print("ACCURACY", BSPredictRF.score(X_test, Y_test))

ACCURACY 0.9398099260823654
